**Car Rental ChatBot**

---



In [1]:
!pip install chatterbot chatterbot-corpus
!pip install pyaml
!pip install spacy
!python -m spacy download en_core_web_sm

!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.5/240.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 85.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import json
from sentence_transformers import SentenceTransformer, util
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer, ChatterBotCorpusTrainer

In [3]:
import random

In [4]:
from google.colab import files
uploaded = files.upload()

Saving qr_car_data_set.json to qr_car_data_set.json


In [8]:
# -------------------------------
# Load dataset
# -------------------------------
dataset = json.load(open(list(uploaded.keys())[0], "r"))

# Shuffle before splitting
random.shuffle(dataset)

# -------------------------------
# Create 80/20 split
# -------------------------------
split_point = int(0.8 * len(dataset))
train_data = dataset[:split_point]
test_data = dataset[split_point:]

print("Training samples:", len(train_data))
print("Testing samples:", len(test_data))

with open(list(uploaded.keys())[0], "r") as f:
    dataset = json.load(f)



Training samples: 439
Testing samples: 110


In [9]:
# from google.colab import files
# uploaded = files.upload()

In [10]:
queries = [item["query"] for item in dataset]
responses = [item["response"] for item in dataset]

In [11]:
model = SentenceTransformer("all-MiniLM-L6-v2")
dataset_embeddings = model.encode(queries, convert_to_tensor=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
bot = ChatBot(
    "CarBot",
    read_only=True,
    logic_adapters=[
        {
            "import_path": "chatterbot.logic.BestMatch",
            "default_response": "Sorry, could you repeat that?",
            "maximum_similarity_threshold": 0.95
        }
    ]
)


In [13]:
trainer = ChatterBotCorpusTrainer(bot)
trainer.train("chatterbot.corpus.english")

Training corpus: 19it [00:03,  6.14it/s]


In [14]:
def smart_response(user_input):

    # Step 1: Try matching with your JSON dataset using MiniLM
    user_emb = model.encode(user_input, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(user_emb, dataset_embeddings)

    best_score = float(cosine_scores.max())
    best_index = int(cosine_scores.argmax())

    if best_score > 0.60:
        return responses[best_index]

    # Step 2: Fall back to ChatterBot for general chat
    try:
        return str(bot.get_response(user_input))
    except:
        return "I'm not sure how to answer that yet."

In [15]:
print("Car Rental ChatBot Ready!")
print("Type 'exit' to quit the chat.")

while True:
    user = input("You: ")

    # Stop the loop if user types 'exit'
    if user.lower() == "exit":
        print("Bot: Goodbye! Have a nice day.")
        break

    reply = smart_response(user)
    print("Bot:", reply)


Car Rental ChatBot Ready!
Type 'exit' to quit the chat.
You: do you have cars
Bot: Here is the list of available cars.
You: available in islamabad
Bot: Honda City 2016, Toyota Revo 2022, Toyota Corolla 2022, Suzuki Alto 2021, Hyundai Tucson 2022 are available in Islamabad.
You: exit
Bot: Goodbye! Have a nice day.


In [16]:
import gradio as gr

# Your existing smart_response function
def smart_response(user_input):
    # Step 1: Dataset similarity using MiniLM
    user_emb = model.encode(user_input, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(user_emb, dataset_embeddings)
    best_score = float(cosine_scores.max())
    best_index = int(cosine_scores.argmax())

    if best_score > 0.80:
        return responses[best_index]

    # Step 2: ChatterBot fallback
    try:
        bot_reply = str(bot.get_response(user_input))
        if bot_reply.strip() == "" or bot_reply == "None":
            return "Can you ask me something about cars, prices, or availability?"
        return bot_reply
    except:
        return "Can you ask me something about cars, prices, or availability?"

# Gradio wrapper
def chatbot_interface(user_query, backend):
    """
    backend parameter exists for UI only; it does not affect functionality.
    """
    try:
        return smart_response(user_query)
    except Exception as e:
        return f"Error: {str(e)}"

# Build the UI
ui = gr.Interface(
    fn=chatbot_interface,
    inputs=[
        gr.Textbox(label="Ask something about cars"),
        gr.Radio(["local", "hf", "groq"], label="Embedding backend", value="local")
    ],
    outputs=gr.Textbox(label="Answer"),
    title="Car Rental Chatbot",
    description="This chatbot answers questions about car availability, pricing, and rentals."
)

ui.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://770c7e60333c961c60.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
